In [52]:
import pandas as pd
import json
LCA_dict=json.loads(open('virushostdb.daily.phage.lca.json', 'r').read())


In [53]:
new_dict={}
df=pd.read_table('virushostdb_output(filtered).tsv', sep='\t')
virus_names=df['virus_name']
for virus_name in virus_names:
    if virus_name in LCA_dict and virus_name not in new_dict:
        new_dict.update({virus_name:LCA_dict[virus_name]})
    else:
        continue


In [54]:
df.head()

,virus_name,viral_acc,evidence,report_host,host_taxid,host_lineage,host_tax_rank,find_name,gcfs,new_gcf,assembly_category,assembly_level,find_taxid,find_taxrank,find_lineage
0,Acholeplasma phage MV-L51,NC_001341,"Literature, RefSeq",Acholeplasma laidlawii,2148,Bacteria; Terrabacteria group; Tenericutes; Mo...,species,Acholeplasma laidlawii,"['GCF_900476025.1', 'GCF_000018785.1']",GCF_900476025.1,Unreference,Complete Genome,2148,species,k__Bacteria;p__Tenericutes;c__Mollicutes;o__Ac...
1,Acholeplasma virus L2,NC_001447,"Literature, NCBI Virus, RefSeq",Acholeplasma laidlawii,2148,Bacteria; Terrabacteria group; Tenericutes; Mo...,species,Acholeplasma laidlawii,"['GCF_900476025.1', 'GCF_000018785.1']",GCF_900476025.1,Unreference,Complete Genome,2148,species,k__Bacteria;p__Tenericutes;c__Mollicutes;o__Ac...
2,Achromobacter phage 83-24,NC_028834,"Literature, RefSeq",Achromobacter xylosoxidans,85698,Bacteria; Proteobacteria; Betaproteobacteria; ...,species,Achromobacter xylosoxidans,['GCF_008432465.1'],GCF_008432465.1,Represent,Complete Genome,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...
3,Achromobacter phage JWAlpha,NC_023556,"Literature, RefSeq",Achromobacter xylosoxidans,85698,Bacteria; Proteobacteria; Betaproteobacteria; ...,species,Achromobacter xylosoxidans,['GCF_008432465.1'],GCF_008432465.1,Represent,Complete Genome,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...
4,Achromobacter phage JWDelta,KF787094,"Literature, NCBI Virus, RefSeq",Achromobacter xylosoxidans,85698,Bacteria; Proteobacteria; Betaproteobacteria; ...,species,Achromobacter xylosoxidans,['GCF_008432465.1'],GCF_008432465.1,Represent,Complete Genome,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...


In [55]:

print(len(df))
print(len(set(df['virus_name'])))
print(len(set(df['report_host'])))
print(len(set(df['find_name'])))


4577
4286
602
527


In [56]:
LCA_dict=new_dict

In [57]:
# 保存
json_string = json.dumps(LCA_dict, indent=4)
with open('virushostdb_output_lca.json', 'w') as f:
    f.write(json_string)


In [58]:
import pandas as pd
table = []
for virus_name in LCA_dict.keys():
    virus_info = LCA_dict[virus_name]
    line_dict = {"virus_name": virus_name, 'virus_id': virus_info['virus_id'], 'host_num': virus_info['host_num'],
                 'lca_level': virus_info['lca_level'], 'lca_name': virus_info['lca_name'], 'lca_lineage': virus_info['lca_lineage']}
    table.append(line_dict)

df_lca = pd.DataFrame(table)
df_lca.to_csv('virushostdb_output_lca.json.tsv', sep='\t', index=False)


In [59]:
from pyecharts.charts import Pie
from pyecharts import options as opts


def draw_pie(df_, title):
    counts = list(zip(df_.iloc[:, 0], df_.iloc[:, 1]))
    word_counts = sorted(counts, key=lambda x: x[1], reverse=True)

    # 绘图
    pie = (
        #创建一个饼图，初始化大小
        Pie(init_opts=opts.InitOpts(width="1000px", height="500px"))
        # 添加数据
        .add(
            "",
            word_counts  # 数据
        )
        # 设置标签格式，b-名称，c是计数，d是概率
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}({d}%)"))
        .set_global_opts(
            #标题
            title_opts=opts.TitleOpts(title="饼图"),
            # 图例的布局朝向。可选：'horizontal', 'vertical'
            legend_opts=opts.LegendOpts(
                orient="vertical", pos_top="0%", pos_right="8%"  # 图例设置
            ),
            #工具栏
            toolbox_opts=opts.ToolboxOpts(
                is_show=True,
                pos_top="bottom",
                pos_left="right",
                feature={
                    "saveAsImage": {},
                    "dataZoom": {"yAxisIndex": "none"},
                    "restore": {},
                    "magicType": {"show": True, "type": ["line", "bar"]},
                    "dataView": {}}
            )
        )

    )
    pie.render(f'{title}.html')  # 保存到本地
    pie.render_notebook()


## lca 统计

In [60]:
prefix = 'virushostdb_output_lca'
# 统计host num
txt = df_lca['host_num'].value_counts()
df_ = pd.DataFrame({'Host Num': txt.index, 'Count': txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
df_.to_csv(f'{prefix}.num.tsv', sep='\t', index=False)
df_


,Host Num,Count,Percent
0,1,4025,0.9391
1,2,213,0.0497
2,3,21,0.0049
3,4,13,0.0030
4,6,6,0.0014
5,5,4,0.0009
6,8,2,0.0005
7,14,1,0.0002
8,7,1,0.0002


In [61]:
draw_pie(df_,f'{prefix}.num')

In [62]:
# 统计lca_level
txt=df_lca['lca_level'].value_counts()
df_ = pd.DataFrame({'Tax level': txt.index, 'Count': txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
df_.to_csv(f'{prefix}.level.tsv', sep='\t', index=False)
df_


,Tax level,Count,Percent
0,species,2902,0.6771
1,strain,1233,0.2877
2,genus,75,0.0175
3,family,64,0.0149
4,order,8,0.0019
5,kingdom,3,0.0007
6,class,1,0.0002


In [63]:
draw_pie(df_, f'{prefix}.level')


In [64]:
# 统计lca_lineage

def get_lineage_rank(x,rank):
    taxonomy_level={'strain':7,'species':6,'genus':5,'family':4,'order':3,'class':2,'phylum':1,'kingdom':0}
    i=taxonomy_level[rank]
    try:
        return x.split(';')[i].split('__')[1]
    except:
        return '-'
f=open(f'{prefix}.lineage.md', 'w')

# kindom
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'kingdom')).value_counts()
df_ = pd.DataFrame({'kingdom':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## kingdom\n')
f.write(df_.to_csv(sep='\t',index=False))
# phylum
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'phylum')).value_counts()
df_ = pd.DataFrame({'phylum':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## phylum\n')
f.write(df_.to_csv(sep='\t',index=False))

# class
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'class')).value_counts()
df_ = pd.DataFrame({'class':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## class\n')
f.write(df_.to_csv(sep='\t',index=False))

# order
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'order')).value_counts()
df_ = pd.DataFrame({'order':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## order\n')
f.write(df_.to_csv(sep='\t',index=False))

# family
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'family')).value_counts()
df_ = pd.DataFrame({'family':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## family\n')
f.write(df_.to_csv(sep='\t',index=False))


# genus
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'genus')).value_counts()
df_ = pd.DataFrame({'genus':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## genus\n')
f.write(df_.to_csv(sep='\t',index=False))


# species
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'species')).value_counts()
df_ = pd.DataFrame({'species':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## species\n')
f.write(df_.to_csv(sep='\t',index=False))


# strain
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'strain')).value_counts()
df_ = pd.DataFrame({'strain':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## strain\n')
f.write(df_.to_csv(sep='\t',index=False))
f.close()

In [ ]:
# 统计lca_lineage

def get_lineage_rank(x,rank):
    taxonomy_level={'strain':7,'species':6,'genus':5,'family':4,'order':3,'class':2,'phylum':1,'kingdom':0}
    i=taxonomy_level[rank]
    try:
        return x.split(';')[i].split('__')[1]
    except:
        return '-'
f=open(f'{prefix}.lineage.md', 'w')

# kindom
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'kingdom')).value_counts()
df_ = pd.DataFrame({'kingdom':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## kingdom\n')
f.write(df_.to_csv(sep='\t',index=False))
# phylum
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'phylum')).value_counts()
df_ = pd.DataFrame({'phylum':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## phylum\n')
f.write(df_.to_csv(sep='\t',index=False))

# class
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'class')).value_counts()
df_ = pd.DataFrame({'class':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## class\n')
f.write(df_.to_csv(sep='\t',index=False))

# order
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'order')).value_counts()
df_ = pd.DataFrame({'order':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## order\n')
f.write(df_.to_csv(sep='\t',index=False))

# family
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'family')).value_counts()
df_ = pd.DataFrame({'family':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## family\n')
f.write(df_.to_csv(sep='\t',index=False))


# genus
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'genus')).value_counts()
df_ = pd.DataFrame({'genus':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## genus\n')
f.write(df_.to_csv(sep='\t',index=False))


# species
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'species')).value_counts()
df_ = pd.DataFrame({'species':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## species\n')
f.write(df_.to_csv(sep='\t',index=False))


# strain
txt=df_lca['lca_lineage'].apply(lambda x: get_lineage_rank(x,'strain')).value_counts()
df_ = pd.DataFrame({'strain':txt.index, 'Count':txt.values})
df_['Percent'] = (df_['Count']/df_['Count'].sum()).round(4)
f.write('## strain\n')
f.write(df_.to_csv(sep='\t',index=False))
f.close()